Let's first define our maze:

Note: an enum is a data type that lets you value code unique items with the unique item being the __name__ and the value called __value__
https://www.youtube.com/watch?v=MO-I8Sun_jw

Our maze is a 10 x 10 2d grid of __Cell__ 's. A __Cell__ is an enum with str values where ' ' represents an empty space and 'X' will represent a blocked space and so forth. It will be represented as lists of cell columns in a list of rows.

In [1]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
from generic_search import dfs, node_to_path, Node, bfs, astar

class Cell(str, Enum):
    EMPTY = ' '
    BLOCKED = 'X'
    START = 'S'
    GOAL = 'G'
    PATH = '*'

class MazeLocation(NamedTuple):
    row: int
    column: int

### Generating a random maze

In [2]:
class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2, \
                start: MazeLocation = MazeLocation(0,0), goal: MazeLocation = MazeLocation(9,9)) -> None:
        #initialise basic instance variables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for col in range(columns)]
                                        for row in range(rows)]
            
        # populate the grid with bloacked cells - method defined below
        # you run this random fill first so your start and goal won't
        # get over written
        self._randomly_fill(rows, columns,sparseness)
        
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
        
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0,1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
    
    # a way to print the maze
    def __str__(self) -> str:
        output: str = ''
        for row in self._grid:
            output += ''.join([col.value for col in row]) + '\n'
        return output
    
    # create a method to check if we have reached the goal cell
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    # create a method that checks the cells/locations around it
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
            
        # restriction on downward movement    
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
            
        # restriction on upward movement    
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        
        # restriction on movement to the right
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        
        # restriction on movement to the left
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        
        return locations

In [3]:
maze: Maze = Maze()
print(maze)

S X   X X 
XX   X  X 
          
  X       
   XX  X X
          
   X  X   
  XXX  XX 
          
 X  XX   G



### Depth-first Search
"A depth-first search (DFS) is what its name suggests: a search that goes as deeply as it can before backtracking to its last decision point if it reaches a dead end. We’ll implement a generic depth-first search that can solve our maze problem."

The algorithm relies on the stack data structure stack. So we add the stack class to our generic_search.py file:

In [4]:
'''
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.pop()  # LIFO

    def __repr__(self) -> str:
        return repr(self._container)
'''

'\nclass Stack(Generic[T]):\n    def __init__(self) -> None:\n        self._container: List[T] = []\n\n    @property\n    def empty(self) -> bool:\n        return not self._container  # not is true for empty container\n\n    def push(self, item: T) -> None:\n        self._container.append(item)\n\n    def pop(self) -> T:\n        return self._container.pop()  # LIFO\n\n    def __repr__(self) -> str:\n        return repr(self._container)\n'

We also want a class that tracks our states/locations. In the case of our maze-solving problem, those states are of type __MazeLocation__. "We’ll call the Node that a state came from its parent. We will also define our Node class as having cost and heuristic properties and with __ lt __ () implemented, so we can reuse it later in the A* algorithm. "

This Node Class let us save each cell/coordinate with a list a attributes:
- State: whether it is a blank, start or goal.
- parent: the previous node

These are useful for tracing the path from the goal back to the start.

In [5]:
'''
class Node(Generic[T]):
    def __init__(self, state: T, parent: Optional[Node], cost: float = 0.0,
     heuristic: float = 0.0) -> None:
        self.state: T = state
        self.parent: Optional[Node] = parent
        self.cost: float = cost
        self.heuristic: float = heuristic

    def __lt__(self, other: Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
'''

'\nclass Node(Generic[T]):\n    def __init__(self, state: T, parent: Optional[Node], cost: float = 0.0,\n     heuristic: float = 0.0) -> None:\n        self.state: T = state\n        self.parent: Optional[Node] = parent\n        self.cost: float = cost\n        self.heuristic: float = heuristic\n\n    def __lt__(self, other: Node) -> bool:\n        return (self.cost + self.heuristic) < (other.cost + other.heuristic)\n'

"An in-progress depth-first search needs to keep track of two data structures: the stack of states (or “places”) that we are considering searching, which we will call the frontier; and the set of states that we have already searched, which we will call explored. As long as there are more states to visit in the frontier, DFS will keep checking whether they are the goal (if a state is the goal, DFS will stop and return it) and adding their successors to the frontier. It will also mark each state that has already been searched as explored, so that the search does not get caught in a circle, reaching states that have prior visited states as successors. If the frontier is empty, it means there is nowhere left to search."

In [6]:
'''
def dfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T],
     List[T]]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    
    # explored is where we've been
    explored: Set[T] = {initial}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node
        
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored:  # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None  # went through everything and never found goal
'''

"\ndef dfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T],\n     List[T]]) -> Optional[Node[T]]:\n    # frontier is where we've yet to go\n    frontier: Stack[Node[T]] = Stack()\n    frontier.push(Node(initial, None))\n    \n    # explored is where we've been\n    explored: Set[T] = {initial}\n\n    # keep going while there is more to explore\n    while not frontier.empty:\n        current_node: Node[T] = frontier.pop()\n        current_state: T = current_node.state\n        # if we found the goal, we're done\n        if goal_test(current_state):\n            return current_node\n        \n        # check where we can go next and haven't explored\n        for child in successors(current_state):\n            if child in explored:  # skip children we already explored\n                continue\n            explored.add(child)\n            frontier.push(Node(child, current_node))\n    return None  # went through everything and never found goal\n"

"If dfs() is successful, it returns the Node encapsulating the goal state. The path from the start to the goal can be reconstructed by working backward from this Node and its priors using the parent property. "

In [7]:
'''
def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    # work backwards from end to front
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path
'''

'\ndef node_to_path(node: Node[T]) -> List[T]:\n    path: List[T] = [node.state]\n    # work backwards from end to front\n    while node.parent is not None:\n        node = node.parent\n        path.append(node.state)\n    path.reverse()\n    return path\n'

Creating Extra methods to the Maze class that let us mark our path and clearing it.

In [8]:
class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2, \
                start: MazeLocation = MazeLocation(0,0), goal: MazeLocation = MazeLocation(9,9)) -> None:
        #initialise basic instance variables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for col in range(columns)]
                                        for row in range(rows)]
            
        # populate the grid with bloacked cells - method defined below
        # you run this random fill first so your start and goal won't
        # get over written
        self._randomly_fill(rows, columns,sparseness)
        
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
        
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0,1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
    
    # a way to print the maze
    def __str__(self) -> str:
        output: str = ''
        for row in self._grid:
            output += ''.join([col.value for col in row]) + '\n'
        return output
    
    # create a method to check if we have reached the goal cell
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    # create a method that checks the cells/locations around it
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
            
        # restriction on downward movement    
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
            
        # restriction on upward movement    
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        
        # restriction on movement to the right
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        
        # restriction on movement to the left
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations        
        
##########new################
    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
        
    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
##########new################


In [9]:
# Test DFS
m: Maze = Maze()
print(m)
print('-----------------------')
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test,
     m.successors)
if solution1 is None:
    print('No solution found using depth-first search!')
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    
    m.clear(path1)

S   X     
      X   
   X      
 X  XX    
   X      
     X    
     X    
 X   X   X
  X    X  
X X      G

-----------------------
S***X*****
   ***X  *
   X  ****
 X  XX*   
   X***   
  ***X    
  *  X    
 X***X***X
  X ***X**
X X      G



### Breath-first search

A breath-first search finds the shortest path but takes way longer than a depth-first search.

To implement BFS, we add a Queue class to our generic_search.py. The data structure know as a queue works on a FIFO basis. We are going to use an efficient python data type called __deque__. __deque__ is a list-like container with fast appends and pops on either end.

Popping from the left on a __deque__ is an O(1) operation, whereas it is an O(n) operation on a __list__. In the case of the __list__, after popping from the left, every subsequent element must be moved one to the left, making it inefficient. 

In [10]:
'''
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
    
    @property
    def empty(self) -> bool:
        return not self._container # not is true for empty container
    
    def push(self, item: T) -> None:
        self._container.append(item)
    
    def pop(self) -> T:
        return self._container.popleft() # FIFO
    
    def __repr__(self) -> str:
        return repr(self.container)
'''

'\nclass Queue(Generic[T]):\n    def __init__(self) -> None:\n        self._container: Deque[T] = Deque()\n    \n    @property\n    def empty(self) -> bool:\n        return not self._container # not is true for empty container\n    \n    def push(self, item: T) -> None:\n        self._container.append(item)\n    \n    def pop(self) -> T:\n        return self._container.popleft() # FIFO\n    \n    def __repr__(self) -> str:\n        return repr(self.container)\n'

BFS:

In [11]:
'''
def bfs(initial: T, goal_test: Callable[[T], bool],\
        successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: Queue[Node[T]] = Queue()
    
    # add the starting point to the queue frontier
    frontier.push(Node(initial,None))
    
    # explored is where we've been
    explored: Set[T] = {initial}
        
    # keep going while there is more to explore
    while not frontier.empty:
        # get the first in the queue to explore
        current_node: Node[T] = frontier.pop()
        # check whether the node is blank, X, S or G
        current_state: T = current_code.state
        
        # remember goal_test is a method in the class maze
        # if current state is equal the goal, we're done
        if goal_test(current_state):
            return current_node
        
        # check where we can go next and haven't explored
        # remember successors is a method in the class maze
        for child in successors(current_state):
            if child in explored:# skip children we already explored
                continue
            # add the new child to the set explored
            explored.add(child)
            
            # add the new child to the queue frontier
            frontier.push(Node(child, current_node))
    return None # gone through everything and never found goal
'''

"\ndef bfs(initial: T, goal_test: Callable[[T], bool],        successors: Callable[[T], List[T]]) -> Optional[Node[T]]:\n    # frontier is where we've yet to go\n    frontier: Queue[Node[T]] = Queue()\n    \n    # add the starting point to the queue frontier\n    frontier.push(Node(initial,None))\n    \n    # explored is where we've been\n    explored: Set[T] = {initial}\n        \n    # keep going while there is more to explore\n    while not frontier.empty:\n        # get the first in the queue to explore\n        current_node: Node[T] = frontier.pop()\n        # check whether the node is blank, X, S or G\n        current_state: T = current_code.state\n        \n        # remember goal_test is a method in the class maze\n        # if current state is equal the goal, we're done\n        if goal_test(current_state):\n            return current_node\n        \n        # check where we can go next and haven't explored\n        # remember successors is a method in the class maze\n        

Testing BFS:

In [12]:
solution2: Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.successors)
if solution2 is None:
    print('No solution found using breadth-first search!')
else:
    path2: List[MazeLocation] = node_to_path(solution2)
    m.mark(path2)
    print(m)
    m.clear(path2)

S   X     
*     X   
*  X      
*X  XX    
*  X      
*    X    
***  X    
 X** X   X
  X*   X  
X X******G



Here's another demo of the BFS algorithm: https://www.youtube.com/watch?v=hettiSrJjM4

### A* Search

An A* search uses a combination of a cost function and a heuristic function to focus its search on pathways most likely to get to the goal quickly.

"The cost function, g(n), examines the cost to get to a particular state. In the case of our maze, this would be how many previous steps we had to go through to get to the state in question. The heuristic function, h(n), gives an estimate of the cost to get from the state in question to the goal state. It can be proved that if h(n) is an admissible heuristic, then the final path found will be optimal. An admissible heuristic is one that never overestimates the cost to reach the goal. On a two-dimensional plane, one example is a straight-line distance heuristic, because a straight line is always the shortest path."

"The total cost for any state being considered is f(n), which is simply the combination of g(n) and h(n). In fact, f(n) = g(n) + h(n). "

The goal is to minimise total cost.

To pick the state with the lowest cost, an A* search algorithm uses a priority queue as a data structure. A priority queue is a queue that is always ordered. For more explanation on priority queue and heaps refer to these two videos:
https://www.youtube.com/watch?v=wptevk0bshY
https://www.youtube.com/watch?v=nXp7lg2SVQo

Python’s standard library contains __heappush()__ and __heappop()__ functions that will take a list and maintain it as a binary heap. We will create a PriorityQueue class using these two functions and add it to __generic_search.py__.

In [13]:
'''
class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        
        # create a list as an empty container
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container # true when container is empty
    
    def push(self, item: T) -> None:
        heappush(self._container, item)
    
    def pop(self) -> T:
        return heappop(self._container)
    
    def __repr__(self) -> str:
        return repr(self._container)
        
'''

'\nclass PriorityQueue(Generic[T]):\n    def __init__(self) -> None:\n        \n        # create a list as an empty container\n        self._container: List[T] = []\n    \n    @property\n    def empty(self) -> bool:\n        return not self._container # true when container is empty\n    \n    def push(self, item: T) -> None:\n        heappush(self._container, item)\n    \n    def pop(self) -> T:\n        return heappop(self._container)\n    \n    def __repr__(self) -> str:\n        return repr(self._container)\n        \n'

#### Heuristics

A heuristic is an intuition for solving a problem.

"Heuristics that calculate smaller values end up leading to a search through more states, whereas heuristics closer to the exact real distance (but not over it, which would make them inadmissible) lead to a search through fewer states. Therefore, ideal heuristics come as close to the real distance as possible without ever going over it."

#### Euclidean Distance

d = sqrt[(x1-x2)^2 + (y1-y2)^2]


In [14]:
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt(xdist**2 + ydist**2)
    return distance

#### Manhattan Distance

The Manhattan distance is the difference between two rows and summing it with the difference between two columns.

In [15]:
def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance

"Because this heuristic more accurately follows the actuality of navigating our mazes (moving vertically and horizontally instead of in diagonal straight lines), it comes closer to the actual distance between any maze location and the goal than Euclidean distance does. Therefore, when an A* search is coupled with Manhattan distance, it will result in searching through fewer states than when an A* search is coupled with Euclidean distance for our mazes. Solution paths will still be optimal, because Manhattan distance is admissible (never overestimates distance) for mazes in which only four directions of movement are allowed."

#### The A* algorithm

We modify the BFS to get our A* search algorithm:

- Make the frontier queue pop nodes with the lowest f(n).
- Change the explored set to a dictionary, which would let us keep track of the lowest cost (g(n)) of each node we may visit. We may visit a node twice because of heuristic.
- Consider every step has a cost of 1.
- Add a heuristic function

In [16]:
'''
def astar(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]],
         heuristic: Callable[[T], float]) -> Optional[Node[T]]:
    
    # frontier is a priority queue containing nodes we would consider
    frontier: PriorityQueue[Node[T]]= PriorityQueue()
    
    # push the first node, start to the frontier
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    
    # explored is where we've been. Make the cost 0 for starting node/cell
    explored: Dict[T, float] = {initial: 0.0}
        
    while not frontier.empty:
        # consider the node at the front of the queue
        current_node: Node[T] = frontier.pop()
        
        # consider the state of this popped node
        current_state: T = current_node.state
        
        # testing for end of maze
        if goal_test(current_state):
            return current_node
        
        # check where we can go next and haven't explored
        for child in successors(current_state):
            new_cost: float = current_node.cost + 1
                # 1 assumes a grid, need a cost function for more sophisticated apps
                
                # add or update node-cost pair to explored

            if child not in explored or explored[child] > new_cost:
                try:
                    print("explored child: ", explored[child], "new cost: ", new_cost)
                except:
                    print("child not in explored, new cost is ", new_cost)      
                explored[child] = new_cost
              
                # push the child into frontier, with all its parent, cost and heuristic details
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None # went through everything and never found goal
'''


"\ndef astar(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]],\n         heuristic: Callable[[T], float]) -> Optional[Node[T]]:\n    \n    # frontier is a priority queue containing nodes we would consider\n    frontier: PriorityQueue[Node[T]]= PriorityQueue()\n    \n    # push the first node, start to the frontier\n    frontier.push(Node(initial, None, 0.0, heuristic(initial)))\n    \n    # explored is where we've been. Make the cost 0 for starting node/cell\n    explored: Dict[T, float] = {initial: 0.0}\n        \n    while not frontier.empty:\n        # consider the node at the front of the queue\n        current_node: Node[T] = frontier.pop()\n        \n        # consider the state of this popped node\n        current_state: T = current_node.state\n        \n        # testing for end of maze\n        if goal_test(current_state):\n            return current_node\n        \n        # check where we can go next and haven't explored\n        for child in su

#### Testing A*

In [17]:
distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)

solution3: Optional[Node[MazeLocation]] = astar(m.start, m.goal_test, m.successors, distance)

if solution3 is None:
    print('No solution found using A*!')
else:
    path3: List[MazeLocation] = node_to_path(solution3)
    m.mark(path3)
    print(m)

child not in explored, new cost is  1.0
child not in explored, new cost is  1.0
child not in explored, new cost is  2.0
child not in explored, new cost is  2.0
child not in explored, new cost is  2.0
child not in explored, new cost is  3.0
child not in explored, new cost is  3.0
child not in explored, new cost is  3.0
child not in explored, new cost is  4.0
child not in explored, new cost is  4.0
child not in explored, new cost is  4.0
child not in explored, new cost is  5.0
child not in explored, new cost is  5.0
child not in explored, new cost is  5.0
child not in explored, new cost is  5.0
child not in explored, new cost is  5.0
child not in explored, new cost is  6.0
child not in explored, new cost is  6.0
child not in explored, new cost is  6.0
child not in explored, new cost is  6.0
child not in explored, new cost is  6.0
child not in explored, new cost is  7.0
child not in explored, new cost is  7.0
child not in explored, new cost is  7.0
child not in explored, new cost is  7.0
